In [44]:
import os
import sys
import pandas as pd
src_dir = os.path.abspath('../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [45]:
import pandas as pd
import asyncio

import gspread_asyncio
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

from datetime import datetime
import pytz
import os

from utils import md_table_to_df


# Файл для сервиcного аккаунта
SRC_DIR = "/Users/V/Desktop/pets/Agro/src"
SERVICE_ACCOUNT_FILE = f'{SRC_DIR}/progressagroproject-9dfb76fd1abc.json'
FOLDER_ID = '1EEbdC6WDtSpy68pmNLHAeU4d7Ca01MdK'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

# Аутентификация
def get_creds():
    return Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )

drive_service = build('drive', 'v3', credentials=get_creds())
# Выносим создание клиента в корень, чтобы ускорить функцию
agcm = gspread_asyncio.AsyncioGspreadClientManager(get_creds)
client = await agcm.authorize()

In [46]:
msk_tz = pytz.timezone('Europe/Moscow')
today_str = datetime.now(msk_tz).strftime('%d.%m.%y')
report_name = f"Отчет_{today_str}"
report_name

'Отчет_17.04.25'

In [47]:
spreadsheet = await client.open(report_name)

In [51]:
file_metadata = await asyncio.to_thread(
    lambda: drive_service.files().get(
        fileId=spreadsheet.id,
#         fileId = '1rqF6Sy2R7WX1RC71Ota_ZUA_h0xKmTMekJDNOx0uhRc',
        fields="parents,trashed"
    ).execute()
)

/usr/local/Cellar/python@3.10/3.10.8/Frameworks/Python.framework/Versions/3.10/lib/python3.10/ast.py:270: RuntimeWarning: coroutine 'apply_conditional_formatting' was never awaited
  for item in field:


HttpError: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/1iikW25jXMbTIQTORm0Bemjv0Hc4vpnGhTrMWiEq-crM?fields=parents%2Ctrashed&alt=json returned "File not found: 1iikW25jXMbTIQTORm0Bemjv0Hc4vpnGhTrMWiEq-crM.". Details: "[{'message': 'File not found: 1iikW25jXMbTIQTORm0Bemjv0Hc4vpnGhTrMWiEq-crM.', 'domain': 'global', 'reason': 'notFound', 'location': 'fileId', 'locationType': 'parameter'}]">

In [49]:
file_metadata

{'trashed': False, 'parents': ['1EEbdC6WDtSpy68pmNLHAeU4d7Ca01MdK']}

In [20]:
if FOLDER_ID not in file_metadata['parents']:
    await asyncio.to_thread(
        lambda: drive_service.files().delete(fileId=spreadsheet.id).execute()
    )

In [50]:
# await asyncio.to_thread(
#         lambda: drive_service.files().delete(fileId=spreadsheet.id).execute()
#     )

''

In [27]:
spreadsheet = await client.open(report_name)
worksheet = await spreadsheet.get_worksheet(0)

TypeError: AsyncioGspreadWorksheet.findall() missing 1 required positional argument: 'query'

In [42]:
from gspread_formatting import *
async def apply_conditional_formatting():
    client = await agcm.authorize()
    spreadsheet = await client.open(report_name)
    worksheet = await spreadsheet.get_worksheet(0)
    # Формируем правило условного форматирования
    request = {
        "requests": [{
            "addConditionalFormatRule": {
                "rule": {
                    "ranges": [{
                        "sheetId": worksheet.id,
                        "startRowIndex": 1,  # Строки 2-1000
                        "endRowIndex": 1000,
                        "startColumnIndex": 0,  # Колонки A-Z
                        "endColumnIndex": 26
                    }],
                    "booleanRule": {
                        "condition": {
                            "type": "CUSTOM_FORMULA",
                            "values": [{"userEnteredValue": "=AND($I2<5, $I2<>\"\")"}]
                        },
                        "format": {
                            "backgroundColor": {"red": 0.99, "green": 0.98, "blue": 0.8}  # Желтый
                        }
                    }
                },
                "index": 0
            }
        }]
    }
    
    await spreadsheet.batch_update(request)

In [43]:
await apply_conditional_formatting()